より実践的な話をしよう

### ベーストラックのバイナリdumpファイルを読む

ベーストラックのバイナリdumpファイルは、C++の形式だと次のように記述されている

```cpp
// base track
double ax, ay, x, y, z;
int pl, isg, zone, dmy;
int64_t rawid;
// micro track 1
double ax1 ay1 z1;
int ph1, pos1, col1, row1, zone1, isg1;
int64_t rawid1;
// micro track 2
double ax2 ay2 z2;
int ph2, pos2, col2, row2, zone2, isg2;
int64_t rawid2;
```
参照 https://gitlab.com/kkodama/NETSCAN-documents/blob/master/vxx-reader.md

Pythonでは、doubleは "d" intは "i" int64_t は "q" なので、ベーストラックの構造体は "dddddiiiiqdddiiiiiiqdddiiiiiiq" と表される

In [ ]:
import struct
import matplotlib.pyplot as plt

#dはdouble
#iは32bit signed int
#qは64bit signed int
struct_fmt = 'dddddiiiiqdddiiiiiiqdddiiiiiiq'
struct_len = struct.calcsize(struct_fmt)
struct_unpack = struct.Struct(struct_fmt).unpack_from

# 読み込む
def read_bbt(file_name):
    datas = []
    with open(file_name, "rb") as f:
        while True:
            data = f.read(struct_len)
            if not data: break
            datas.append(data)
    return datas

# 出力する
def write_bbt(file_name, datas):
    with open(file_name, "wb") as f: 
        for data in datas:
            f.write(data)

# ファイルの読み込み
datas1= read_bbt("b001.bbt")
print("# of tracks = ",len(datas1))

# ファイルの出力
write_bbt("b001-.bbt", datas1)


#### リスト生成の文法

```python
vx1 = []
for data in datas1:
    vx1.append(struct_unpack(data)[2])
```
と
```python
vx1 = [struct_unpack(data)[2] for data in datas1]
```
は同じ意味になる。行数を減らせるので、次のコードで使う。

In [ ]:
# XYの飛跡密度分布を描画
vx1 = [struct_unpack(data)[2] for data in datas1]
vy1 = [struct_unpack(data)[3] for data in datas1]
plt.hist2d(vx1,vy1, bins=[20,20])
plt.colorbar()
plt.title("X-Y")
plt.show()

# AX-AYの飛跡密度分布を描画
vax1 = [struct_unpack(data)[0] for data in datas1]
vay1 = [struct_unpack(data)[1] for data in datas1]
plt.hist2d(vax1,vay1, bins=[100,100],range=[[-0.1,0.1],[-0.1,0.1]])
plt.colorbar()
plt.title("AX-AY")
plt.show()

# PHV分布
vphv = [(struct_unpack(data)[13]+struct_unpack(data)[23])%10000 for data in datas1]
plt.hist(vphv,range=[0,200],bins=100)
plt.title("PHV")
plt.show()

# PH分布
vph = [(struct_unpack(data)[13]+struct_unpack(data)[23])//10000 for data in datas1]
plt.hist(vph,range=[0,33],bins=33)
plt.title("PH")
plt.show()
